In [1]:
pip install tensorflow==2.10

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Read the text file
with open('Project-III/data/data/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

2024-06-24 10:31:37.707920: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 10:31:38.163474: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-24 10:31:38.163654: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-06-24 10:31:38.227355: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-24 10:31:39.307603: W tensorflow/stream_executor/pla

Tokenize

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

Padding

In [5]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))
print(total_words)

8200


Neural Network

In [8]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

2024-06-24 10:31:44.461518: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-06-24 10:31:44.462033: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-06-24 10:31:44.462077: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-192-12-22): /proc/driver/nvidia/version does not exist
2024-06-24 10:31:44.464019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2,208,800
Trainable params: 2,208,800
Non-trainable params: 0
_________________________________________________________________
None


Training

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

2024-06-24 10:31:44.874615: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3159099200 exceeds 10% of free system memory.


Epoch 1/100
3010/3010 [==============================] - 95s 31ms/step - loss: 6.2433 - accuracy: 0.0763
Epoch 2/100
3010/3010 [==============================] - 92s 31ms/step - loss: 5.5190 - accuracy: 0.1217
Epoch 3/100
3010/3010 [==============================] - 93s 31ms/step - loss: 5.1301 - accuracy: 0.1466
Epoch 4/100
3010/3010 [==============================] - 93s 31ms/step - loss: 4.8028 - accuracy: 0.1637
Epoch 5/100
3010/3010 [==============================] - 93s 31ms/step - loss: 4.4959 - accuracy: 0.1824
Epoch 6/100
3010/3010 [==============================] - 94s 31ms/step - loss: 4.2075 - accuracy: 0.2034
Epoch 7/100
3010/3010 [==============================] - 92s 31ms/step - loss: 3.9293 - accuracy: 0.2277
Epoch 8/100
3010/3010 [==============================] - 92s 31ms/step - loss: 3.6695 - accuracy: 0.2589
Epoch 9/100
3010/3010 [==============================] - 92s 30ms/step - loss: 3.4246 - accuracy: 0.2921
Epoch 10/100
3010/3010 [==============================]

Generate our predicitons

In [11]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 31ms/step
I will leave if they should be in
